In [1]:
import requests
import pandas as pd
import json
import numpy as np
from datetime import datetime, date
import csv
import time
import ssl
ssl._create_default_https_context = ssl._create_unverified_context
pd.options.mode.chained_assignment = None  # default='warn'

In [2]:
def get_twse_stock_db_info():
    link = 'http://www.twse.com.tw/exchangeReport/BWIBBU_ALL?response=open_data'
    df = pd.read_csv(link, encoding='utf_8_sig')
    return df
    
def get_twse_stock_info(df, stock):
    target_data = df[df["股票代號"] == int(stock)]
    name = target_data.iloc[0]['股票名稱']
    priceEarningRatio = target_data.iloc[0]['本益比']
    yieldRatio = target_data.iloc[0]['殖利率(%)']
    priceBookRatio = target_data.iloc[0]['股價淨值比']
    name, priceEarningRatio, yieldRatio, priceBookRatio
    return name, priceEarningRatio, yieldRatio, priceBookRatio

def get_otc_stock_db_info():
    link = 'http://www.tpex.org.tw/openapi/v1/tpex_mainboard_peratio_analysis'
    json_data = requests.get(link).json()
    df = pd.DataFrame.from_records(json_data)
    return df
    
def get_otc_stock_info(df, stock):
    target_data = df[df['SecuritiesCompanyCode'] == stock]
    name = target_data.iloc[0]['CompanyName']
    priceEarningRatio = target_data.iloc[0]['PriceEarningRatio']
    dividendPerShare = target_data.iloc[0]['DividendPerShare']
    yieldRatio = target_data.iloc[0]['YieldRatio']
    priceBookRatio = target_data.iloc[0]['PriceBookRatio']
    name, priceEarningRatio, yieldRatio, priceBookRatio
    return name, priceEarningRatio, yieldRatio, priceBookRatio

In [3]:
def string_with_comma_to_int(x):
    return int(x.replace(",", ""))

def string_with_comma_to_float(x):
    try:
        return float(x.replace(",", ""))
    except:
        return 0
    
def string_to_float(x):
    try:
        return float(x)
    except:
        return 0
    
def vol_for_twse(x):
    try:
        return round(float(x.replace(",", ""))/1000)
    except:
        return 0

def moving_average(x, w):
    return np.convolve(x, np.ones(w), "valid") / w

def get_stock_volumn_price(yy, mm, dd, stock_tag):
    date_tag = date_tag = str(yy) + str(mm).zfill(2) + "01"
    link = 'http://www.twse.com.tw/exchangeReport/STOCK_DAY?response=open_data&date=%s&stockNo=%s'%(date_tag, stock_tag)
    try:
        df = pd.read_csv(link, encoding='utf_8_sig')
    except:
        return None
    df_target = df[['日期', '成交股數', '收盤價']]
    df_target.iloc[:, 1] = df_target.iloc[:, 1].apply(vol_for_twse) # volumn
    df_target.iloc[:, 2] = df_target.iloc[:, 2].apply(string_to_float) # price
    return df_target

def get_otc_stock_volumn_price(yy, mm, dd, stock_tag):
    yy = yy - 1911
    json_data = requests.get('http://www.tpex.org.tw/web/stock/aftertrading/daily_trading_info/st43_result.php?d=%s/%s/%s&stkno=%s'%(yy, mm, dd, stock_tag)).json()
    columns = ['日期', '成交股數', '成交金額', '開盤價', '最高價', '最低價', '收盤價', '漲跌價差', '成交筆數']
    df = pd.DataFrame(json_data['aaData'], columns=columns)
    df_target = df[['日期', '成交股數', '收盤價']]
    df_target.iloc[:, 1] = df_target.iloc[:, 1].apply(string_with_comma_to_float) # volumn
    df_target.iloc[:, 2] = df_target.iloc[:, 2].apply(string_with_comma_to_float) # price
    return df_target

In [4]:
def get_time_duration_stock_info(time_list, stock_tag, min_volumn = 150, ma_num = 5, isOtc = False):
    df_all = pd.DataFrame()
    for time_data in time_list:
        yy = time_data[0]
        for mm in range(time_data[1], time_data[2] + 1):
            if isOtc:
                df = get_otc_stock_volumn_price(yy, mm, "01", stock_tag)
            else:
                df = get_stock_volumn_price(yy, mm, "01", stock_tag)
                if df is None:
                    continue
                time.sleep(0.15)
            df_all = pd.concat([df_all, df], axis=0)
            
    df_np = df_all.to_numpy().copy()
    
    if len(df_np) == 0:
        return None, None
    # if df_np[:, 1].max() > min_volumn:
    #     df_np[:, 1][df_np[:, 1] < min_volumn] = min_volumn
    # if df_np[:, 1].max() == df_np[:, 1].min():
    #     return None, None
    # if df_np[:, 2].max() == df_np[:, 2].min():
    #     return None, None
    
    # df_np[:, 1] = (df_np[:, 1] - df_np[:, 1].min()) / ((df_np[:, 1].max() - df_np[:, 1].min()))
    # df_np[:, 2] = (df_np[:, 2] - df_np[:, 2].min()) / ((df_np[:, 2].max() - df_np[:, 2].min()))
    
    # df_vol_ma = moving_average(df_np[:, 1], ma_num)
    # df_pri_ma = moving_average(df_np[:, 2], ma_num)
    df_vol_ma = df_np[:, 1]
    df_pri_ma = df_np[:, 2]
    
    vol_data = df_vol_ma
    pri_data = df_pri_ma
    return vol_data, pri_data

def calculate_target(vol_data, pri_data):
    aa = -7
    bb = -28
    if vol_data[bb:aa].mean() == 0:
        return False
    if pri_data[bb:aa].mean() == 0:
        return False
    try:
        # vol_valid = vol_data[aa:].mean() / vol_data[bb:aa].mean() > 5
        # vol_valid = np.median(vol_data[aa:]) / np.median(vol_data[bb:aa]) > 7
        vol_med_factor = 0.5
        vol_cur_factor = 0.5
        vol_short = ((1 - vol_cur_factor) * (vol_med_factor * np.median(vol_data[aa:]) + (1 - vol_med_factor) * np.mean(vol_data[aa:])) + vol_cur_factor * vol_data[-1])
        # vol_short = (vol_med_factor * np.median(vol_data[aa:]) + (1 - vol_med_factor) * np.mean(vol_data[aa:]))
        vol_long = (vol_med_factor * np.median(vol_data[bb:aa]) + (1 - vol_med_factor) * np.mean(vol_data[bb:aa]))
        vol_valid = vol_short / vol_long > 4.3
        vol_valid = vol_valid and (vol_short > 800)
        # previous_high_valid = pri_data[-10:].max() / pri_data[-1] <= 1.08
        # previous_high_valid = previous_high_valid & (pri_data[-20:].max() / pri_data[-1] < 1.05)
        # pri_valid = pri_data[aa:].mean() / pri_data[bb:aa].mean() > 1.03
        # pri_ma = moving_average(pri_data, 4)
        # trend_valid =  pri_ma[-8:].mean()/pri_ma[-88:-80].mean() > 0.8
        pri_5ma = moving_average(pri_data, 5)
        pri_10ma = moving_average(pri_data, 10)
        pri_20ma = moving_average(pri_data, 20)
        pri_60ma = moving_average(pri_data, 60)
        # trend_valid = (pri_5ma[-1]/pri_10ma[-1] > 0.98) and (pri_5ma[-1]/pri_20ma[-1] > 1.01) and (pri_5ma[-1]/pri_60ma[-1] > 1.02)
        ma_pos_valid = (pri_data[-1] / pri_5ma[-1] >= 0.95) and (pri_data[-1] / pri_10ma[-1] >= 0.98) and (pri_data[-1] / pri_20ma[-1] >= 1.00) and (pri_data[-1] / pri_60ma[-1] >= 1.00)
        ma_close_valid = (pri_5ma[-1]) / (pri_10ma[-1]) <= 1.08 and (pri_5ma[-1]) / (pri_10ma[-1]) > 0.94
        ma_close_valid = ma_close_valid and (pri_10ma[-1]) / (pri_20ma[-1]) <= 1.07 and (pri_10ma[-1]) / (pri_20ma[-1]) > 0.95
    except:
        return False
    # print(vol_valid, trend_valid, ma_pos_valid)
    return vol_valid and ma_pos_valid and ma_close_valid


def calculate_target_trend(vol_data, pri_data):
    data_min_num = 70
    if len(vol_data) < data_min_num or len(pri_data) < data_min_num:
        # print("Not enough data amount")
        return False
    aa = -7
    bb = -28
    if vol_data[bb:aa].mean() == 0:
        return False
    if pri_data[bb:aa].mean() == 0:
        return False
    
    pos_valid = True
    vol_valid = True
    trend_valid = True
    
    try:
        vol_valid = vol_valid and np.max(vol_data[aa:-1]) > 600
        pri_cur = pri_data[-1]
        pri_5ma = moving_average(pri_data, 5)
        pri_10ma = moving_average(pri_data, 10)
        pri_20ma = moving_average(pri_data, 20)
        pri_60ma = moving_average(pri_data, 60)
        
        pos_valid = pos_valid and pri_cur / pri_5ma[-1] <= 1.04 and pri_cur / pri_5ma[-1] >= 0.97
        pos_valid = pos_valid and pri_cur / pri_10ma[-1] <= 1.04 and pri_cur / pri_10ma[-1] >= 0.97
        pos_valid = pos_valid and pri_5ma[-1] / pri_10ma[-1] <= 1.04 and pri_5ma[-1] / pri_10ma[-1] >= 0.975
        pos_valid = pos_valid and pri_10ma[-1] / pri_20ma[-1] <= 1.06 and pri_10ma[-1] / pri_20ma[-1] >= 0.98
        pos_valid = pos_valid and pri_cur / pri_20ma[-1] <= 1.07 and pri_cur / pri_20ma[-1] >= 1
        pos_valid = pos_valid and pri_10ma[-1] / pri_60ma[-1] >= 1.04
        
        trend_valid = trend_valid and pri_20ma[-1] / pri_20ma[-8] >= 1.025
        trend_valid = trend_valid and pri_60ma[-1] / pri_60ma[-10] >= 1.03
    except:
        # print("?")
        return False
    return vol_valid and pos_valid and trend_valid

In [5]:
# trend_valid = trend_valid and pri_20ma[-1] / pri_20ma[-8] >= 1.025
# trend_valid = trend_valid and pri_60ma[-1] / pri_60ma[-2] >= 1.035

In [6]:
# target_list = []
# time_list = [
#     # [2023, 6, 9],
#     [2023, 12, 12],
#     [2024,  1,  4],
#     # [2023,  7,  10],
# ]

# vol_data, pri_data = get_time_duration_stock_info(time_list, '1319', min_volumn=150, ma_num=1, isOtc=False)
# valid = calculate_target_trend(vol_data, pri_data)

In [7]:
# df = get_twse_stock_db_info()
# get_twse_stock_info(df, "2330")

# df = get_otc_stock_db_info()
# get_otc_stock_info(df, "1785")

In [8]:
stock_list = []
data_path = r".\db\twse.csv"
isOtc = False
# data_path = r".\db\otc.csv"
# isOtc = True
with open(data_path, newline='', encoding='utf_8_sig') as csvfile:
    line_list = csv.reader(csvfile)
    for line in line_list:
        stock_list.append(line[0])

In [9]:
target_list = []
time_list = [
    # [2022, 3, 7],
    [2023, 12, 12],
    [2024,  1,  4],
    # [2023,  7,  10],
]

if isOtc:
    df = get_otc_stock_db_info()
else:
    df = get_twse_stock_db_info()

for stock in stock_list:
    vol_data, pri_data = get_time_duration_stock_info(time_list, stock, min_volumn=150, ma_num=1, isOtc=isOtc)
    if vol_data is None or pri_data is None:
        continue
    # valid = calculate_target(vol_data, pri_data)
    valid = calculate_target_trend(vol_data, pri_data)
    if valid:
        if isOtc:
            name, priceEarningRatio, yieldRatio, priceBookRatio = get_otc_stock_info(df, stock)
            target_list.append([stock, name, priceEarningRatio, yieldRatio, priceBookRatio])
            print(stock, name, priceEarningRatio, yieldRatio, priceBookRatio)
        else:
            name, priceEarningRatio, yieldRatio, priceBookRatio = get_twse_stock_info(df, stock)
            target_list.append([stock, name, priceEarningRatio, yieldRatio, priceBookRatio])
            print(stock, name, priceEarningRatio, yieldRatio, priceBookRatio)

1319 東陽 24.12 3.25 2.86
1442 名軒 9.85 7.54 2.36
1504 東元 20.47 3.89 1.49
1535 中宇 16.53 4.44 2.51
1615 大山 18.32 4.25 3.26
1618 合機 19.48 2.55 1.49
1773 勝一 29.79 1.7 5.4
2020 美亞 8.07 10.32 2.12
2211 長榮鋼 15.23 4.85 2.3
2316 楠梓電 9.59 1.13 0.9
2382 廣達 27.65 3.16 5.89


In [ ]:
with open(r"D:\Stock\result\20240411_trend.csv", 'a', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerows(target_list)